In [ ]:
#Please cite us if you find this code useful:
#Currently under peer review, but this doi: pre-print is live.
#https://doi.org/10.1101/2020.06.25.171918    

# DeepGANnel A Convolutional Generative Adversarial Network to generate labelled timeseries physiological data

### Import libraries  Tested with TensorFlow 2.0

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from functools import partial

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
print("to here")

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display
import sys
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import random as rd
from datetime import datetime
import csv 

In [ ]:
# If you use too many files it can "confuse" the network and lead to the generation of average data
files=["claire_035[NF=50Hz]0-32assembled.csv"]

seq=1280
jumps=seq
limit=5000000000000000

sam=False #are we importing real or synthetic data from Sam.
CARTOON=True # draw a cartoon?
baselinecorrect=False
scamp=10
flip=True # simple augmentation parameters
masternoise=1.0
SD=[]

#plot
start=15000
end=25000

def mean(list):
    return(sum(list)/len(list))

print('Parsing data...')
def dofile(newfile):
    print(newfile)
    i=0
    j=0
    firstline=True
    x=[]
    y=[]
    with open(newfile) as infile:
        reader = csv.reader(infile)
        for row in reader:

            if firstline==True:
                # Don't read first line if header exists
                firstline=False
                continue

            elif i > limit:
                # Stop if past limit
                break

            else:
                # Add current and channels to respective lists
                i += 1
                try:
                    x.append(float(row[1]))
                except:
                    print("An exception occurred",print(row[1]))
                    break
                y.append(float(row[2]))

            if i % 10000 == 0:
                # Progress update every 10000 iterations
                print(f'Progress: {i}/{limit}', end='\r')
                sys.stdout.flush()
        #make sure y is zero or 1
        miny=min(y)
        maxy=max(y)
        print('max and mins of labels',maxy, miny)
        zy=[]
        for val in y:
            if val == miny:
                zy.append(0)
            else:
                zy.append(1)
        return x,zy
x,y = [],[]
for filename in files:
    newx,newy=dofile(filename)
    print('\nnewx len = \n',np.asarray(newx).shape)
    x.extend(newx)
    y.extend(newy)
    print('x shape, y shape',np.asarray(x).shape,np.asarray(y).shape)
    
print("\nlength of lists=\n")         
len(x)
num=len(x) # Number of samples in x and y

#reverse the lists and concatenate
superx=[val for val in x]
supery=[val for val in y]
x.reverse()
y.reverse()
superx.extend([val for val in x])
supery.extend([val for val in y])
x=superx
y=supery
print("\nlength of lists now=\\")    
print('x shape',np.asarray(x).shape)
num=len(superx) # Number of samples in x and y

In [ ]:
# Do some scaling
x=np.asarray(x)
y=np.asarray(y)

maxx=np.max(x)
maxy=np.max(y)
miny=np.min(y)
minx=np.min(x)
maxer=max([maxx,maxy])
miner=max([minx,miny])


print("original global max and mins", maxer,miner)

y=y*2
y=y-1

x=x-miner
x=x/(maxer)-0.5

print("\nx",np.min(x),np.max(x),end="\n")
print("\ny",np.min(y),np.max(y),end="\n")
#should now be +/- 1 or less!!!
print('Converting to correct shape...')
'''offset x!!'''
from scipy.stats import mode
x=x.tolist()
y=y.tolist()
vals=[val for val in x if val < 0]
mode,count= mode(vals)
x=[val-mode for val in x]
del vals

plt.subplot(2,1,1)
plt.plot(x[start:end])

plt.subplot(2,1,2)
plt.plot(y[start:end])

plt.show()


In [ ]:
def noiseup(listin,label,numberofcopies,noiselev):
    # We start by creating a large augmented dataset based on the original
    # The examples will be inperceptable different to each other (by human eye)
    nc=numberofcopies
    copies=[el for el in listin]
    l_label=[el for el in label]
    signal=np.asarray(listin)
    for k in range(nc):
        sd=rd.uniform(noiselev/2, noiselev*2)
        print("noise loop:{:} = {:} ".format(k,sd))
        noise = np.random.normal(0, sd, signal.shape)
        copies.extend(list(signal + noise))
        #must duplicated y too!
        label.extend(l_label)
        start=15000
        end=25000
    return copies, label

#make 10 noise copies
x,y=noiseup(x,y,200,0.003)
print("\nlength of lists post noising=\n")         
print(len(x),len(y))

newx=[el for el in x]

plt.subplot(2,1,1)
plt.plot(x[start:end])

plt.subplot(2,1,2)
plt.plot(y[start:end])

plt.show()

In [ ]:
def makeShape2c(x, y, seq, num, i):
    z = np.ones((2,seq,1))
    z[0, 0:seq, 0] = x[i: i+seq]
    z[1, 0:seq, 0] = y[i: i+seq]
    
    if i % (seq * 10000) == 0:
        print(f'Progress: Batch {i // seq}/{num // seq}', end='\r')
        sys.stdout.flush()

    return z

'''Now draw the labels and data out
note 2c is just a code for the version of this function we used'''
mtrain_images=[makeShape2c(x, y, seq, num, i) for i in range(0, num-seq, jumps)]


In [ ]:
#To balnce the data set (we don't want pages of nothing) only keep those with between f1 and f2 zeros... eg, 0.1, 0.9
# We will treat LIKE images, but of course "image" has no meaning, it is just an array
if 'x' in locals() or 'y' in globals():
    del x
    del y
pmin=0.1
pmax=0.9
better=[]
print("images.shape",np.asarray(mtrain_images).shape)
print("Now deleting boring images")
for image in mtrain_images:
    lmin=min(image[1,:,0])
    array=np.asarray(image)
    mins=array[1,array[1,:,0]==lmin,0] #how many zeros
    z=len(list(mins))
    p=z/len(image[1,:,0])
    if p>0.1 and p<0.9:
        better.append(image)
        
    #if max(image[1,:,0])!=min(image[1,:,0]):
        #better.append(image)''
mtrain_images=better
print("images.shape",np.asarray(mtrain_images).shape)
        

print('\nConverting to numpy array')
mtrain_images=np.asarray(mtrain_images)

sys.stdout.flush()
print(f'\n{mtrain_images.shape}\n')
print('finito')

In [ ]:
#print(len(x),len(y))
mtrain_images = mtrain_images.reshape(mtrain_images.shape[0],2,seq,1)
print(mtrain_images.shape, np.min(mtrain_images),np.max(mtrain_images))

In [ ]:
BUFFER_SIZE = mtrain_images.shape[0]
BATCH_SIZE = 128
looser=[]
glooser=[]
ms=[]
dtw=[]
xout=[] 
yout=[]
ep=[]
EPOCHS = 1000000

In [ ]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(mtrain_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Two models are needed, one Generator and one Discriminator

## The Generator

In [ ]:
def make_generator_model():
    # Up scale through each layer, starting with a small strip of 2xX data
    model = tf.keras.Sequential()
    model.add(layers.Dense(2*10*64, use_bias=False, input_shape=(2*seq,),dtype='float32'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((2, 10, 64)))
    assert model.output_shape == (None, 2, 10, 64) # Bbatch size is "None"

    model.add(layers.Conv2DTranspose(64, (2, 5), strides=(1, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2, 20, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())                                                                                                      
    
    model.add(layers.Conv2DTranspose(64, (2, 5), strides=(1, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2, 40, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(32, (2, 5), strides=(1, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2, 80, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU()) 
    
    model.add(layers.Conv2DTranspose(32, (2, 5), strides=(1, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2, 160, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())          
    
    model.add(layers.Conv2DTranspose(32, (2, 50), strides=(1, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2, 320, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(32, (2, 5), strides=(1, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2, 640, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())        
       
    model.add(layers.Conv2DTranspose(1, (2, 5), strides=(1, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 2, seq, 1)
    model.summary()
    return model   

In [ ]:
generator = make_generator_model()

In [ ]:
# Generate a random block of data the right shape
noise = tf.random.normal([1, 2*seq])
generated_data = generator(noise, training=False)

## The Discriminator

A simple CNN binary classifier. Is it real of fake?
The discriminator will classify generated data as real or fake. 
**Positive = real** data and **negative = fake**.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    #start with your 2xseq THEN DOWN SAMPLE BEFORE MAKING THE FINAL CALL.
    model.add(layers.Conv2D(64, (2, 100), strides=(1, 2), padding='same',
                                     input_shape=[2, seq, 1],dtype='float32'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (2, 50), strides=(1, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (2, 25), strides=(1, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(128, (2, 5), strides=(1, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (2, 5), strides=(1, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(512, (2, 5), strides=(1, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
discriminator.summary()
decision = discriminator(generated_data)

## Define the losses

We need one function for each model.

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

## Discriminator loss

In [ ]:
def discriminator_loss(real_output, fake_output):
    # if the discriminator (or gen bad) did well, all real are 1, loss =0
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    
    # If discriminator did well or gen bad, will be zero
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

## Generator loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

## Define the optimzers

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(3e-6)
discriminator_optimizer = tf.keras.optimizers.Adam(3e-7)

## Save updates
In case of interrupts, should automatically restart. Code care of Google TensorFlow tutorial

In [ ]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop



In [ ]:
noise_dim =2*seq
num_examples_to_generate = 3

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])
noise = tf.random.normal([BATCH_SIZE, noise_dim],stddev=masternoise,seed=7)

def compute_pairwise_distances(x, y):
  """Computes the squared pairwise Euclidean distances between x and y.
  Args:
    x: a tensor of shape [num_x_samples, num_features]
    y: a tensor of shape [num_y_samples, num_features]
  Returns:
    a distance matrix of dimensions [num_x_samples, num_y_samples].
  Raises:
    ValueError: if the inputs do no matched the specified dimensions.
  """

  if not len(x.get_shape()) == len(y.get_shape()) == 2:
    raise ValueError('Both inputs should be matrices.')

  if x.get_shape().as_list()[1] != y.get_shape().as_list()[1]:
    raise ValueError('The number of features should be the same.')

  norm = lambda x: tf.reduce_sum(tf.square(x), 1)

  # By making the `inner' dimensions of the two matrices equal to 1 using
  # broadcasting then we are essentially substracting every pair of rows
  # of x and y.
  # x will be num_samples x num_features x 1,
  # and y will be 1 x num_features x num_samples (after broadcasting).
  # After the substraction we will get a
  # num_x_samples x num_features x num_y_samples matrix.
  # The resulting dist will be of shape num_y_samples x num_x_samples.
  # and thus we need to transpose it again.
  return tf.transpose(norm(tf.expand_dims(x, 2) - tf.transpose(y)))


def gaussian_kernel_matrix(x, y, sigmas):
  r"""Computes a Guassian Radial Basis Kernel between the samples of x and y.
  We create a sum of multiple gaussian kernels each having a width sigma_i.
  Args:
    x: a tensor of shape [num_samples, num_features]
    y: a tensor of shape [num_samples, num_features]
    sigmas: a tensor of floats which denote the widths of each of the
      gaussians in the kernel.
  Returns:
    A tensor of shape [num_samples{x}, num_samples{y}] with the RBF kernel.
  """

  x=tf.cast(x,dtype=tf.float32)
  y=tf.cast(y,dtype=tf.float32)
  beta = 1. / (2. * (tf.expand_dims(sigmas, 1)))

  dist = compute_pairwise_distances(x, y)

  s = tf.matmul(beta, tf.reshape(dist, (1, -1)))

  return tf.reshape(tf.reduce_sum(tf.exp(-s), 0), tf.shape(dist))


In [ ]:
#import utils

sigmas = [
      1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
      1e3, 1e4, 1e5, 1e6
  ]
gaussian_kernel = partial(
    gaussian_kernel_matrix, sigmas=tf.constant(sigmas))


def maximum_mean_discrepancy(x, y,kernel=gaussian_kernel):

    r"""Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.

    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of
    the distributions of x and y. Here we use the kernel two sample estimate
    using the empirical mean of the two distributions.

    MMD^2(P, Q) = || \E{\phi(x)} - \E{\phi(y)} ||^2
    = \E{ K(x, x) } + \E{ K(y, y) } - 2 \E{ K(x, y) },

    where K = <\phi(x), \phi(y)>,
    is the desired kernel function, in this case a radial basis kernel.

    Args:
    x: a tensor of shape [num_samples, num_features]
    y: a tensor of shape [num_samples, num_features]
    kernel: a function which computes the kernel in MMD. Defaults to the
    GaussianKernelMatrix.

    Returns:
    a scalar denoting the squared maximum mean discrepancy loss.
    """
    #with tf.name_scope('MaximumMeanDiscrepancy'):
        # \E{ K(x, x) } + \E{ K(y, y) } - 2 \E{ K(x, y) }
    cost = tf.reduce_mean(kernel(x, x))
    cost += tf.reduce_mean(kernel(y, y))
    cost -= 2 * tf.reduce_mean(kernel(x, y))

    # We do not allow the loss to become negative.
    cost = tf.where(cost > 0, cost, 0, name='value')
    return cost



In [ ]:
# Since this is a compiled function it is difficult to modify on the fly.
@tf.function
def train_step(images,train_disc):
    noise = tf.random.normal([BATCH_SIZE, noise_dim],stddev=masternoise)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)
      #could Dr the data here to binarise the idl row
      # and even do something else..
      #how many real images does the descriminator think are correct?
      real_output = discriminator(images, training=train_disc)
      # So if the DISCRIMINATOR is perfect, it will return 1(s).
   
      # How many fake images does the discriminator detect?
      fake_output = discriminator(generated_images, training=train_disc)
      # So if DISCRIMINATOR is perfect, this would be all -1s
      # if DISCRIMINATOR is poor this will be all 1s
      # ...but of course if GENERATOR is perfect it will also be 1s
      # So I am thinking now; if we rip out a generated_data and find that that the idealisation says just one number
      # when rounded. probably that was no good so should multiply the losses by some factor

      gen_loss = generator_loss(fake_output)
      # if the data were perfect (or discriminator awful) this will be tiny.
      disc_loss = discriminator_loss(real_output, fake_output)
      # if gen bad or discrim great, this is going to be zero
            
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    return disc_loss, gen_loss

In [ ]:
def newLR(LR, midlr, LRbase, LRmax):
    a=1
    '''do nothing just testing'''


In [ ]:
def train(dataset, epochs,startep=0):
  import random
  global looser, glooser
  loss=0
  gloss=0
  train_disc=True
  noise=False
  flip = False
  mu=0
  dlr=3e-7
  glr=1e-5

  sigma=0.0001
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
        if noise==True:
          #Add noise Clearly should use a tf.random not python!
          for image in image_batch:
            b=tf.dtypes.cast(tf.random.normal([seq], mean=mu, stddev=sigma),tf.float64)
            a=tf.dtypes.cast(image[0, :, 0],tf.float64)
            a=tf.math.add(a,b)
            b=tf.dtypes.cast(image[1, :, 0],tf.float64)
            image=tf.stack([a,b])
            #Shape has changed from (2,seq,1) to (2,seq)
        if flip == True:
            for image in image_batch:
              if random.randint(1, 2)==1:
                image = tf.image.flip_left_right(image)
        loss, genl= train_step(image_batch,train_disc)
        if genl>10*loss:
            #deactivated this now
            train_disc=True
        else:
            train_disc=True

    glooser.append(genl.numpy().item())   
    looser.append(loss.numpy().item())
    
    # Produce images for a GIF as we go
    # Save the model every x epochs
    interceptor=50
    if (epoch + 1) % interceptor == 0:
      glr_up=False
      if sum(looser[-interceptor:])/len(looser[-interceptor:])<0.2:
        dlr= tf.keras.backend.get_value(discriminator_optimizer.learning_rate)*.75
        tf.keras.backend.set_value(discriminator_optimizer.learning_rate,dlr)
      elif sum(looser[-interceptor:])/len(looser[-interceptor:])>1:
        dlr= tf.keras.backend.get_value(discriminator_optimizer.learning_rate)*1.5
        tf.keras.backend.set_value(discriminator_optimizer.learning_rate,dlr) 
      if sum(glooser[-interceptor:])/len(glooser[-interceptor:])<0.2:
        glr= tf.keras.backend.get_value(generator_optimizer.learning_rate)*.99
        tf.keras.backend.set_value(generator_optimizer.learning_rate,glr)
      elif sum(glooser[-interceptor:])/len(glooser[-interceptor:])>3:
        glr_up=True
        glr= tf.keras.backend.get_value(generator_optimizer.learning_rate)*1.01
        if glr >1e-2:
            glr=1e-2
        tf.keras.backend.set_value(generator_optimizer.learning_rate,glr)        
    
      checkpoint.save(file_prefix = checkpoint_prefix)
      # Generate after the final epoch
      display.clear_output(wait=True)
      generate_and_save_images(generator,
                           int(epoch+startep),
                           seed)
      print("Train discriminator was ",train_disc)
      with open('somefile.txt', 'a') as the_file:
        the_file.write('Glr_up {}, Time for epoch {} is {:.0f} sec, DLR={:.2e}, GLR={:.2e}, loss {:.4f} gloss {:.2f}\n'.format(glr_up,
           epoch + startep,time.time()-start,dlr, glr, sum(looser[-interceptor:])/len(looser[-interceptor:]),
            sum(glooser[-interceptor:])/len(glooser[-interceptor:])))
      if len (looser)>interceptor:
        print ('Glr_up {}, Time for epoch {} is {:.0f} sec, DLR={:.2e}, GLR={:.2e}, loss {:.4f} gloss {:.2f}'.format(glr_up,
           epoch + startep,time.time()-start,dlr, glr, sum(looser[-interceptor:])/len(looser[-interceptor:]),
            sum(glooser[-interceptor:])/len(glooser[-interceptor:])))
      else:
        print ('Time for epoch {} is {:.0f} sec, loss {:.4f} gloss {:.2f}'.format(epoch + startep,
          time.time()-start,sum(looser)/len(looser),sum(glooser)/len(glooser)))

**Generate and save images**



In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.  
  # This is so all layers run in inference mode (batchnorm).
  # want to pick a different set each time I do
  global xout, yout, ep, SD
  num_examples_to_generate=4 #caution this is a REDEFINITION
  crop=200
  seconds = seconds = int(time.time()-start)
  tf.random.set_seed(seconds)
  seed = tf.random.normal([num_examples_to_generate, noise_dim], stddev=1)
  offset = False
  test_input=seed
  predictions = model(test_input, training=False)
  x=tf.convert_to_tensor(mtrain_images[0:num_examples_to_generate,:,:,0].reshape(num_examples_to_generate,2*1280))
  y=tf.reshape(predictions[0:num_examples_to_generate,:,:,0],shape=(num_examples_to_generate,2*1280))

  #tf.convert_to_tensor(data_np, np.float32)
  #print(maximum_mean_discrepancy(x,y))
  #ms.append(maximum_mean_discrepancy(x,y))
  
  #x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
  #y = np.array([[2,2], [3,3], [4,4]])
  #fastdist, path = fastdtw(x, y, dist=euclidean)
  #print("Dynamic Time Warping metric:")
  #print(fastdist)
  #dtw.append(fastdist)
  fig = plt.figure(figsize=(12,4))
  seqfig=seq-(crop*2)
  xax = range(seqfig*predictions.shape[0])
  data=[]
  labs=[]
  for i in range(predictions.shape[0]):
      newdata=predictions[i, 0, crop:seq-crop, 0] #do some cropping to clear glitch
      newlabs=predictions[i, 1, crop:seq-crop, 0]
      newlabs=np.asarray(newlabs)
      newlabs=np.round(1.5+newlabs/2)
      newlabs=list(newlabs)       
      if offset:
          # Offset to match-up the zero levels
          zeros=[i for i, e in enumerate(newlabs) if e == 2]
          if len(zeros)>0:
              offers=np.asarray(newdata)
              offset=np.mean(offers[zeros])
              # offset=sum(data[zeros])/len(zeros)
              newdata=[e-offset for e in newdata]  
      data.extend(newdata)
      labs.extend(newlabs)
      
      #plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
  if (epoch-1)%20 == 0:
    xout.extend([tens.numpy() for tens in data])
    yout.extend(labs)
    ep.extend(len(data) * [epoch-1])
  realdat=[]
  reallab=[]
  for im in range(num_examples_to_generate):
     limage=rd.choice(range(len(mtrain_images)))
     realdat.extend(mtrain_images[limage,0, crop:seq-crop, 0])
     reallab.extend(mtrain_images[limage,1, crop:seq-crop, 0])
  reallab=[1.5+val/2 for val in reallab]
  #realdat=[2*val for val in realdat]
  if CARTOON==False:  
      plt.subplot(4,1,1)
      plt.plot(xax,realdat)
      plt.plot(xax,reallab)  
      plt.subplot(4,1,2)
      plt.plot(xax,data)
      plt.plot(xax,np.round(labs))
      plt.hlines(0, 0, seqfig*predictions.shape[0],linestyles='dashed')
      vs=[seqfig,2*seqfig,3*seqfig]
      plt.vlines(vs,-0.5,2,linestyles='dashed')  
      #plt.ylim([-1,2.1])
      plt.axis('on')
      plt.subplot(4,1,3)
      plt.plot(range(len(looser)),looser,'o')
      plt.plot(range(len(glooser)), np.asarray(glooser)/4, 'o')
      #print("prediction SD = {}".format(np.std(np.asarray([el for el in data if el < 0]))))
      sdtemp=[el for el in data if el > 0]
      sdtemp=np.asarray(sdtemp)
      sdtemp=np.std(sdtemp).item() 
      SD.append(sdtemp)
      plt.subplot(4,1,4)
      plt.plot(range(len(SD)),SD)
  else:
      plt.subplot(2,1,1)
      plt.plot(xax,realdat)
      plt.plot(xax,reallab)  
      plt.hlines(0, 0, seqfig*predictions.shape[0],linestyles='dashed')
      plt.subplot(2,1,2)
      plt.plot(xax,data)
      plt.plot(xax,np.round(labs))
      plt.hlines(0, 0, seqfig*predictions.shape[0],linestyles='dashed')
      plt.axis('on')
  
  plt.savefig("images/images_at_epoch_{:06d}.png".format(epoch),jpg=120)
  plt.show()
  
  if epoch==EPOCHS-1:
    np.savetxt('mmd.csv', ms, delimiter=',')
    np.savetxt('dtw.csv', dtw, delimiter=',')
    np.savetxt('mmd.csv', ms, delimiter=',')
    out=np.vstack([xout,yout,ep])
    print("outshape",out)
    np.savetxt('synth.csv', out.T, delimiter=',')


## Train!!
Sometimes necessary to tinker with the training rates! Losses should balance, but generator loss always much greater when running well.

In [ ]:
looser=[]
glooser=[]

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
tf.keras.backend.set_value(discriminator_optimizer.learning_rate,3e-7) 
tf.keras.backend.set_value(generator_optimizer.learning_rate,3e-6) 

train(train_dataset, EPOCHS)

## Create a GIF/MOVIES for demonstration purposes


###### Use `imageio` to create an animated gif using the images saved during training.

In [ ]:
'''anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I',fps=2) as writer:
  filenames = glob.glob('images/image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)'''

In [ ]:
anim_file = 'dcgan.mp4'
with imageio.get_writer(anim_file,fps=20) as writer:
  filenames = glob.glob('images/image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

anim_file = 'dcgan.gif'
with imageio.get_writer(anim_file,fps=4) as writer:
  filenames = glob.glob('images/image*.png')
  filenames = sorted(filenames)
  decimate=2
  for count,filename in enumerate(filenames):
    if count%decimate==0:
        image = imageio.imread(filename)
        writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)
    
    


## Collate data for output

In [ ]:
np.savetxt('loss.csv', looser, delimiter=',')
np.savetxt('gloss.csv', glooser, delimiter=',')
np.savetxt('mmd.csv', ms, delimiter=',')
np.savetxt('dtw.csv', dtw, delimiter=',')
np.savetxt('mmd.csv', ms, delimiter=',')
out=np.vstack([xout,yout,ep])
np.savetxt('synth.csv', out.T, delimiter=',')


In [ ]:
print(np.vstack([xout,yout,ep]))

In [ ]:
filenames = glob.glob('images/image*.png')

In [ ]:
filenames

In [ ]:
print("images/images_at_epoch_{:06d}.png".format(15900+300))

In [ ]:
## from PIL import Image, ImageDraw, ImageSequence, ImageFont
import io
anim_file = 'dcganlabout.mp4'
im = Image.open('dcganlab.gif')
transparent = (255,0,0,0)
bkg = Image.new('RGBA',(200,100),transparent)
# A list of the frames to be output
font = ImageFont.truetype("calibri.ttf", 25)
frames = []
i=0

for frame in ImageSequence.Iterator(im):
    i+=1
    frame = frame.convert('RGBA')
    # Draw the text on the frame
    im.paste(bkg,(110,40))
    d = ImageDraw.Draw(frame)
    d.text((110,40), "Real idealisation", fill=(0,0,0),font=font)
    d.text((110,100), "Real raw data", fill=(0,0,0),font=font)
    d.text((110,160), "GAN idealisation", fill=(0,0,0),font=font)
    d.text((110,220), "GAN raw data", fill=(0,0,0),font=font)    


    del d

    b = io.BytesIO()
    frame.save(b, format="GIF")
    frame = Image.open(b)

    # Then append the single frame image to a list of frames
    frames.append(frame)
# Save the frames as a new image
frames[0].save('GANTEXT.gif', save_all=True, append_images=frames[1:])

    

In [ ]:
import ffmpy
ff = ffmpy.FFmpeg(
inputs={'GANTEXT.gif': None},
    outputs={'GANTEXT.mp4': None})
ff.run()

In [ ]:
import moviepy.editor as mp

clip = mp.VideoFileClip("GANTEXT.gif")
clip.write_videofile("GANTEXT.mp4")